<a href="https://colab.research.google.com/github/PrintTrd/elgo_data_pipeline/blob/main/scripts/Preprocess_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sales Dataset Processing

ข้อมูล Business Sales Dataset ที่ได้มาจากคนรู้จักนั้น มีทั้งหมด 14 tables ซึ่งมาจาก database ที่ยังไม่นิ่ง ไม่ได้ผ่านการออกแบบมาดี มีความซ้ำซ้อนอยู่มาก และรับข้อมูลมาจากหลายช่องทางการขาย/platform รวมกัน หลังจากดูข้อมูลคร่าว ๆ ก็ได้ไปคุยกับเจ้าของข้อมูลเพื่อทำความเข้าใจมากขึ้น

## Data integration
เนื่องจากมีความซ้ำซ้อนอยู่มากจึงต้อง normalize ข้อมูล โดยเริ่มจากการไปช่วยออกแบบ ER diagrams ให้ใหม่ก่อน เพื่อวางแผนหน้าตาโครงสร้างข้อมูลใหม่ที่เราควรจะแปลงไปด้วย

หลังจากออกแบบใหม่ก็จะเลือกตัดส่วนที่ดูไม่เกี่ยวข้องกับการขายออกไปบางส่วน เช่น table inventory_input ที่เกี่ยวข้องกับการจัดการ stock สินค้า เพื่อนำมาใช้กับโปรเจคนี้ และข้อมูลบางส่วนก็จะไม่ได้ normalize จนเป็นหน่วยเล็กที่สุด ออกแบบจนได้ ER diagram เป้าหมายออกมาดังภาพ

![](https://drive.google.com/uc?export=view&id=15IVQGpYuX6pkeUbAQVbwUNkN5HxD7I5E)
จากภาพ บางคอลัมน์ที่ยังไม่มีการเก็บข้อมูลเพิ่มเข้ามาก็จะข้ามไปก่อน

## Data Integration Plan Steps

1. Extract and rename tables

2. Transform - Schema/Value Integration <br>
- Rename columns
- Change format and add day
- รวม data
- แยก data
- สร้าง fake values
- สรุปผลข้อมูล
- Check Value

3.   Load - Insert data into MySQL Database
     - อ่าน Secrets ของ Colab (เก็บลง class Config)
     - ใช้ Sqlalchemy เชื่อมต่อไปที่ MySQL
     - สร้าง Schema (database)
     - Insert ข้อมูลลงใน tables

4.   Query ข้อมูลใน table และเซฟ output
     - Query ข้อมูลด้วย Pandas
     - Output ไฟล์เป็น parquet

5.   ทดลองอ่านไฟล์ parquet เพื่อตรวจสอบ



# Step 1) Download and extract

##  Rename tables
- Geocoding -> address
-  item_inventory -> inventory
- product_code -> product
- color_code -> color
- material_code -> material
- size_code -> size

โหลดไฟล์ข้อมูล csv มาใส่ Notebook จากนั้นย้ายที่เก็บ และเปลี่ยนชื่อใหม่

In [2]:
!mv 'elgo_source_database - sale_order.csv' sale_order.csv
!mv 'elgo_source_database - cn_order.csv' canceled_order.csv
!mv 'elgo_source_database - inventory_output.csv' order_item.csv
!mv 'elgo_source_database - cn_item.csv' canceled_item.csv

!mv 'elgo_source_database - awb_info.csv' waybill.csv
!mv 'elgo_source_database - Geocoding.csv' address.csv
!mv 'elgo_source_database - customer_code.csv' customer_code.csv

!mv 'elgo_source_database - item_inventory.csv' inventory.csv
!mv 'elgo_source_database - product_code.csv' product_code.csv
!mv 'elgo_source_database - color_code.csv' color_code.csv
!mv 'elgo_source_database - material_code.csv' material_code.csv
!mv 'elgo_source_database - size_code.csv' size_code.csv
!mv 'elgo_source_database - Price.csv' price.csv

## Read CSV

สร้าง Dataframes

In [123]:
import polars as pl
df = {}
csv_list = ["sale_order", "canceled_order", "order_item", "canceled_item", "waybill", "address", "customer_code", "inventory", "product_code", "color_code", "material_code", "size_code", "price"]
for file_name in csv_list:
  # read csv
  df[f'{file_name}'] = pl.read_csv(f'{file_name}.csv', has_header=True, infer_schema_length=10000, null_values=["COMPUTED_VALUE"])
  # remove all null columns
  df[f'{file_name}'] = df[f'{file_name}'][[column.name for column in df[f'{file_name}'] if not (column.null_count() == df[f'{file_name}'].height)]]
  # remove all null rows
  df[f'{file_name}'] = df[f'{file_name}'].filter(~pl.all_horizontal(pl.all().is_null()))


In [124]:
df

{'sale_order': shape: (1_468, 23)
 ┌──────────┬───────────────┬───────────────┬────────────┬───┬──────┬──────┬───────────────┬────────┐
 │ id       ┆ record date   ┆ sell out date ┆ invoice no ┆ … ┆ note ┆ pdf  ┆ shiping_statu ┆ Status │
 │ ---      ┆ ---           ┆ ---           ┆ ---        ┆   ┆ ---  ┆ ---  ┆ s             ┆ ---    │
 │ str      ┆ str           ┆ str           ┆ str        ┆   ┆ str  ┆ str  ┆ ---           ┆ str    │
 │          ┆               ┆               ┆            ┆   ┆      ┆      ┆ str           ┆        │
 ╞══════════╪═══════════════╪═══════════════╪════════════╪═══╪══════╪══════╪═══════════════╪════════╡
 │ 65272231 ┆ 6/17/2023     ┆ 6/17/2023     ┆ EV01/01    ┆ … ┆ null ┆ done ┆ done          ┆ null   │
 │          ┆ 15:36:55      ┆ 15:36:55      ┆            ┆   ┆      ┆      ┆               ┆        │
 │ 574db9e2 ┆ 6/17/2023     ┆ 6/17/2023     ┆ 85/16      ┆ … ┆ null ┆ done ┆ done          ┆ null   │
 │          ┆ 16:31:53      ┆ 16:31:53      ┆   

## Drop and rename columns

### sale_order
  - id -> order_id
  - record date ->  updated_at
  - sell out date -> created_at
  - invoice no2 -> invoice_number
  - order number -> order_number
  - sales channel -> sales_channel
  - require vat -> require_vat
  - shipping charges -> shipping_charge
  - discount bath -> discount_baht
  - customer code -> customer_category_id
  - customer name -> customer_name
  - customer tax id -> tax_id
  - customer address -> billing_address
  - shipping address -> shipping_address
  - customer tel -> phone
  - customer email -> email
  - Status -> status

In [113]:
df["sale_order"]

id,record date,sell out date,invoice no,invoice no2,invoice folder,invoice page,order number,sales channel,require vat,shipping charges,discount bath,customer code,customer name,customer tax id,customer address,shipping address,customer tel,customer email,note,pdf,shiping_status,Status
str,str,str,str,str,str,i64,str,str,bool,i64,i64,str,str,str,str,str,str,str,str,str,str,str
"""65272231""","""6/17/2023 15:3…","""6/17/2023 15:3…","""EV01/01""","""IV2306A0016""","""EV01""",1,null,"""line""",true,0,0,"""EC0004""",null,null,null,null,null,null,null,"""done""","""done""",null
"""574db9e2""","""6/17/2023 16:3…","""6/17/2023 16:3…","""85/16""","""IV2306A0017""","""85""",16,"""230617KE2PKF8F…","""shopee""",true,0,0,"""EC0002""",null,null,null,null,null,null,null,"""done""","""done""",null
"""547d27f1""","""6/17/2023 16:3…","""6/17/2023 16:3…","""85/17""","""IV2306A0021""","""85""",17,"""230617M1WTBSHV…","""shopee""",true,0,0,"""EC0002""",null,null,null,null,null,null,null,"""done""","""done""",null
"""79d68bd1""","""6/18/2023 14:0…","""6/18/2023 14:0…","""85/18""","""IV2306A0022""","""85""",18,"""71721995487644…","""lazada""",true,0,0,"""EC0001""",null,null,null,null,null,null,null,"""done""","""done""",null
"""b2c5be32""","""6/18/2023 17:4…","""6/18/2023 17:4…","""85/19""","""IV2306A0023""","""85""",19,"""230618PMVTBWM4…","""shopee""",true,0,0,"""EC0002""",null,null,null,null,null,null,null,"""done""","""done""",null
"""2e07ac0f""","""6/19/2023 18:4…","""6/19/2023 18:4…","""85/20""","""IV2306A0018""","""85""",20,"""230619RHJ9GM1V…","""shopee""",true,0,0,"""EC0002""",null,null,null,null,null,null,null,"""done""","""done""",null
"""0f604387""","""6/20/2023 9:00…","""6/20/2023 9:00…","""85/21""","""IV2306A0024""","""85""",21,"""230619SSDJ3QMF…","""shopee""",true,0,0,"""EC0002""",null,null,null,null,null,null,null,"""done""","""done""",null
"""85808b44""","""6/20/2023 9:01…","""6/20/2023 9:01…","""85/22""","""IV2306A0025""","""85""",22,"""71798070159575…","""lazada""",true,0,0,"""EC0001""",null,null,null,null,null,null,null,"""done""","""done""",null
"""198605b6""","""6/20/2023 22:1…","""6/20/2023 22:1…","""85/23""","""IV2306A0020""","""85""",23,"""230620UXM4QBAM…","""shopee""",true,0,0,"""EC0002""",null,null,null,null,null,null,null,"""done""","""done""",null


In [125]:
df["sale_order"] = df["sale_order"].drop(columns=['invoice no', 'invoice folder', 'invoice page', 'note', 'pdf', "shiping_status"])

In [126]:
df["sale_order"] = df["sale_order"].rename({
    "id": "order_id",
    "record date": "updated_at",
    "sell out date": "created_at",
    "invoice no2": "invoice_number",
    "order number": "order_number",
    "sales channel": "sales_channel",
    "require vat": "require_vat",
    "shipping charges": "shipping_charge",
    "discount bath": "discount_baht",
    "customer code": "customer_category_id",
    "customer name": "name",
    "customer tax id": "tax_id",
    "customer address": "billing_address",
    "shipping address": "shipping_address",
    "customer tel": "phone",
    "customer email": "email",
    "Status":"status"
})
df["sale_order"].columns

['order_id',
 'updated_at',
 'created_at',
 'invoice_number',
 'order_number',
 'sales_channel',
 'require_vat',
 'shipping_charge',
 'discount_baht',
 'customer_category_id',
 'name',
 'tax_id',
 'billing_address',
 'shipping_address',
 'phone',
 'email',
 'status']

### canceled_order
  - id -> canceled_id: ไว้เชื่อมกับ cn_item ก่อน
  - creditnote no -> updated_at
  - invoice id -> invoice_number
  - note -> cancel_reason
  - ...นอกนั้นคล้ายที่แก้ให้ sale_order ไป...

In [117]:
df["canceled_order"]

id,creditnote no,issue date,order_id,invoice id,invoice folder,invoice page,order number,sales channel,require vat,shipping charges,discount bath,customer code,customer name,customer tax id,customer address,shipping address,customer tel,customer email,note,pdf,shiping_status,Status,image
str,str,str,str,str,str,str,str,str,bool,i64,i64,str,str,str,str,str,i64,str,str,str,str,str,str
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""93339d0c""","""01/03/2024 00:…","""01/03/2024""","""ee066971""","""87/88-CN""",null,null,null,"""nursing home""",true,0,125,null,"""EH0072""",null,"""65 Thanon Witt…",null,923750555,null,"""ออกบิลใหม่เนื่…",null,null,null,"""cn_order_Image…"
"""77f53cb8""","""29/02/2024 00:…","""03/03/2024""","""094636c1""","""93/02-CN""",null,null,"""240218V0RUWE5H…","""shopee""",true,0,0,null,"""Prem Singh""",null,null,"""11 kasem samra…",66967755727,null,"""เบอร์ผิด""",null,null,null,"""cn_order_Image…"
"""5a03509a""","""29/02/2024 00:…","""03/03/2024""","""1f870eca""","""93/19-CN""",null,null,"""2402215U02C6XB…","""shopee""",true,0,0,null,"""Prem Singh""",null,null,"""11 kasem samra…",66967755727,null,"""เบอร์ผิด""",null,null,null,"""cn_order_Image…"
"""9a0e9066""","""03/03/2024 00:…","""03/03/2024""","""2b79b368""","""93/55-CN""",null,null,"""82547889311676…","""lazada""",true,0,0,null,"""สคุ นธา ชยั ฤก…",null,"""สคุ นธา ชยั ฤก…","""26/1 หม ู่13 ·…",660896015539,null,"""ลูกค้าขอยกเลิก…",null,null,null,"""cn_order_Image…"
"""ae6876dc""","""10/03/2024 00:…","""10/03/2024""","""2b79b368""","""93/55-CN""",null,null,"""82547889311676…","""lazada""",true,0,0,null,"""สคุ นธา ชยั ฤก…",null,"""สคุ นธา ชยั ฤก…","""26/1 หม ู่13 ·…",660896015539,null,"""ลูกค้าต้องการย…",null,null,null,"""cn_order_Image…"
"""6cee400d""","""13/03/2024 00:…","""13/03/2024""","""deb9a927""","""-CN""",null,null,"""2403034CWQ6837…","""shopee""",true,0,0,null,"""ชัชวาลย์ นพรัก…",null,null,"""หมู่บ้านเดอะเก…",66943456449,null,"""ไม่มีคนรับสาย""",null,null,null,"""cn_order_Image…"
"""01d7aa97""","""20/03/2024 00:…","""20/03/2024""","""4219b2ad""","""IV2403A0059-CN…",null,null,"""24031557TVKF2R…","""shopee""",true,0,0,null,"""ชนากานต์ ด้วงก…",null,null,"""91 หมู่ 12, ตํ…",66984146969,null,"""ไม่มีผู้รับ""",null,null,null,"""cn_order_Image…"


In [127]:
df["canceled_order"] = df["canceled_order"].drop(columns=['image', 'issue date'])
df["canceled_order"].columns

['id',
 'creditnote no',
 'order_id',
 'invoice id',
 'order number',
 'sales channel',
 'require vat',
 'shipping charges',
 'discount bath',
 'customer name',
 'customer address',
 'shipping address',
 'customer tel',
 'note']

In [129]:
df["canceled_order"] = df["canceled_order"].rename({
    "id": "cancel_id",
    "creditnote no": "updated_at",
    "invoice id": "invoice_number",
    "order number": "order_number",
    "sales channel": "sales_channel",
    "require vat": "require_vat",
    "shipping charges": "shipping_charge",
    "discount bath": "discount_baht",
    "customer name": "name",
    "customer address": "billing_address",
    "shipping address": "shipping_address",
    "customer tel": "phone",
    "note":"cancel_reason"
})
df["canceled_order"].columns

['cancel_id',
 'updated_at',
 'order_id',
 'invoice_number',
 'order_number',
 'sales_channel',
 'require_vat',
 'shipping_charge',
 'discount_baht',
 'name',
 'billing_address',
 'shipping_address',
 'phone',
 'cancel_reason']

### order_item
  - id -> order_item_id
  - sell out date -> created_at : จะได้รู้ราคาในเวลานั้นได้เร็ว
  - order id -> order_id
  - quantity (pack) -> quantity_pack
  - price per pack -> unit_price
  - price per pack (ex vat) -> unit_price_ex_vat
  - total amount -> total_price

In [63]:
df["order_item"].columns

['id',
 'sell out date',
 'order id',
 'master product code',
 'master product name',
 'readable name',
 'product code',
 'color code',
 'size code',
 'price per pack',
 'pcs per pack',
 'quantity (pack)',
 'quantity',
 'price per pack (ex vat)',
 'total amount']

In [130]:
df["order_item"] = df["order_item"].drop(columns=['master product name', 'readable name', 'product code','color code','size code', 'pcs per pack', 'quantity'])

In [131]:
df["order_item"] = df["order_item"].rename({
    "id": "order_item_id",
    "sell out date": "created_at",
    "order id": "order_id",
    "master product code": "master_product_code",
    "price per pack": "unit_price",
    "price per pack (ex vat)": "unit_price_ex_vat",
    "quantity (pack)": "quantity_pack",
    "total amount": "total_price"
})

In [132]:
df["order_item"].columns

['order_item_id',
 'created_at',
 'order_id',
 'master_product_code',
 'unit_price',
 'quantity_pack',
 'unit_price_ex_vat',
 'total_price']

In [133]:
df["order_item"]

order_item_id,created_at,order_id,master_product_code,unit_price,quantity_pack,unit_price_ex_vat,total_price
str,str,str,str,f64,i64,f64,f64
"""70427678""","""6/17/2023 15:3…","""65272231""","""EP-012-F-01-A0…",1100.0,2,1028.037383,2200.0
"""9c69cd6d""","""6/17/2023 15:4…","""65272231""","""EP-011-F-01-A0…",880.0,2,822.429907,1760.0
"""3082b336""","""6/17/2023 16:3…","""574db9e2""","""EP-006-M-02-A0…",0.0,1,0.0,0.0
"""d56b8678""","""6/17/2023 16:3…","""574db9e2""","""EP-021-F-01-A0…",0.0,1,0.0,0.0
"""11a252de""","""6/17/2023 16:3…","""547d27f1""","""EP-006-M-02-A0…",0.0,1,0.0,0.0
"""f81865d5""","""6/18/2023 14:0…","""79d68bd1""","""EP-022-5-01-A0…",0.0,1,0.0,0.0
"""67bdb19b""","""6/18/2023 17:4…","""b2c5be32""","""EP-001-M-01-A0…",0.0,1,0.0,0.0
"""285b6a0e""","""6/19/2023 18:4…","""2e07ac0f""","""EP-001-M-01-A0…",0.0,1,0.0,0.0
"""100b680b""","""6/19/2023 18:4…","""2e07ac0f""","""EP-021-F-01-A0…",0.0,1,0.0,0.0


### canceled_item
  - id -> order_item_id
  - cn id -> order_id
  - ...นอกนั้นคล้าย inventory_output แต่ข้อมูล created_at ต้องเติมเองอีกที...

In [134]:
df["canceled_item"]

id,cn id,readable name,master product code,product code,size code,color code,price per pack,pcs per pack,quantity (pack),total amount
str,str,str,str,str,str,str,i64,i64,i64,i64
"""1dcc08c8""","""93339d0c""","""7f75f77c""","""EP-004-L-01-A0…","""EP-004""","""L""","""01-A000""",2500,1,1,2500
"""03563dac""","""77f53cb8""","""f2f05ac4""","""EP-011-F-01-A0…","""EP-011""","""F""","""01-A021""",880,2,1,880
"""cbd09247""","""5a03509a""","""45e38e8e""","""EP-011-F-01-A0…","""EP-011""","""F""","""01-A021""",880,2,1,880
"""f5944f71""","""9a0e9066""","""a67ba987""","""EP-011-F-01-A0…","""EP-011""","""F""","""01-A009""",460,1,1,460
"""9cfcb6b6""","""ae6876dc""","""a67ba987""","""EP-011-F-01-A0…","""EP-011""","""F""","""01-A009""",460,1,1,460
"""b58d766b""","""6cee400d""","""c35ff230""","""EP-006-XL-02-A…","""EP-006""","""XL""","""02-A001""",1750,1,1,1750
"""358fdfaf""","""01d7aa97""","""c6e50079""","""EP-022-5-01-A0…","""EP-022""","""5""","""01-A000""",100,5,1,100
"""230795b7""","""618c4e45""","""2cad9158""","""EP-007-F-01-A0…","""EP-007""","""F""","""01-A005""",480,1,1,480
"""562ba118""","""8676de51""","""bd75d1ae""","""EP-011-F-01-A0…","""EP-011""","""F""","""01-A002""",460,1,1,460


In [135]:
df["canceled_item"].columns

['id',
 'cn id',
 'readable name',
 'master product code',
 'product code',
 'size code',
 'color code',
 'price per pack',
 'pcs per pack',
 'quantity (pack)',
 'total amount']

In [136]:
df["canceled_item"] = df["canceled_item"].drop(columns=['readable name', 'product code','color code','size code', 'color code', 'pcs per pack'])

In [137]:
df["canceled_item"] = df["canceled_item"].rename({
    "id": "order_item_id",
    "cn id": "order_id",
    "master product code": "master_product_code",
    "price per pack": "unit_price",
    "quantity (pack)": "quantity_pack",
    "total amount": "total_price"
})
df["canceled_item"].columns

['order_item_id',
 'order_id',
 'master_product_code',
 'unit_price',
 'quantity_pack',
 'total_price']

### waybill
  - ordernumber -> order_number
  - address -> shipping_address

In [138]:
df["waybill"].columns

['ordernumber',
 'file',
 'platform',
 'name',
 'address',
 'billing_address',
 'phone',
 'price',
 'postcode',
 'ship_date',
 'tracking_number',
 'invoice_number',
 'awb_format',
 'sales_channel_id']

In [139]:
df["waybill"] = df["waybill"].drop(columns=['file', 'platform', 'price', 'sales_channel_id'])
df["waybill"].columns

['ordernumber',
 'name',
 'address',
 'billing_address',
 'phone',
 'postcode',
 'ship_date',
 'tracking_number',
 'invoice_number',
 'awb_format']

In [140]:
df["waybill"] = df["waybill"].rename({
    "ordernumber": "order_number",
    "address": "shipping_address"
})
df["waybill"].columns

['order_number',
 'name',
 'shipping_address',
 'billing_address',
 'phone',
 'postcode',
 'ship_date',
 'tracking_number',
 'invoice_number',
 'awb_format']

### customer_code
  - code -> customer_category_id
  - customer address -> shipping_address
  - billing address -> billing_address
  - tax id -> tax_id
  - tel -> phone
  - description -> note
  - channel catagory -> channel_catagory

In [82]:
df["customer_code"]

code,category,number,name,customer address,billing address,branch,tax id,tel,email,contact person,description,channel catagory,require tax
str,str,i64,str,str,str,str,str,str,str,str,str,str,bool
"""A002""","""A""",2,"""บ.มหาชัยฟู๊ดส์…","""71/11 หมู่ที่6…",null,"""00001""","""0105532104629""",null,null,null,"""ซื้อน้ำมัน pal…","""Food Factory""",true
"""A015""","""A""",15,"""บริษัท เจดีฟู้…","""48/52-53 ม.1 ถ…",null,"""00001""","""0107563000151""",null,null,null,null,"""Food Factory""",true
"""A016""","""A""",16,"""บริษัท เจดีฟู้…","""116, 116/1, 11…",null,"""00001""","""0107563000151""",null,null,null,null,"""Food Factory""",true
"""A001""","""A""",1,"""บ.แอลไลด์เคมีเ…","""255/2 นิคมอุตส…",null,"""00001""","""0105534105169""",null,null,null,null,"""Food Factory""",true
"""A042""","""A""",42,"""บ.ทริปเปิ้ล-เอ…","""314/15,314/16 …",null,"""สำนักงานใหญ่""","""0105548105522""",null,null,null,null,"""Food Factory""",true
"""A046""","""A""",46,"""บ.อินทัชธนากร …","""216/2 ม.1 ต.บ้…",null,"""สาขาที่ 1""","""0105540071343""",null,null,null,null,"""Food Factory""",true
"""A047""","""A""",47,"""บ.เอ็น เอส ที …","""45/19 ม.5 ซ.วั…",null,"""-""","""0105540011871""",null,null,null,null,"""Food Factory""",true
"""A048""","""A""",48,"""บ.เอส ทรี พี โ…","""513/2 ถนนประชา…",null,"""สำนักงานใหญ่""","""0105560171564""",null,null,null,null,"""Food Factory""",true
"""A049""","""A""",49,"""บ.สินสวัสดิ์แท…","""204 ม.6 ถ.สุขุ…",null,null,"""0115534001789""",null,null,null,null,"""Food Factory""",true


In [141]:
df["customer_code"] = df["customer_code"].drop(columns=['category', 'number', 'contact person', 'require tax'])
df["customer_code"].columns

['code',
 'name',
 'customer address',
 'billing address',
 'branch',
 'tax id',
 'tel',
 'email',
 'description',
 'channel catagory']

In [142]:
df["customer_code"] = df["customer_code"].rename({
    "code": "customer_category_id",
    "customer address": "shipping_address",
    "billing address": "billing_address",
    "tax id": "tax_id",
    "tel": "phone",
    "description": "note",
    "channel catagory": "channel_catagory"
})
df["customer_code"].columns

['customer_category_id',
 'name',
 'shipping_address',
 'billing_address',
 'branch',
 'tax_id',
 'phone',
 'email',
 'note',
 'channel_catagory']

# Step 2) Transform - Schema/Value Integration

Process ข้อมูลด้วย Polars ในการรวมและแยก Dataset
- เติมและย้ายข้อมูลให้ถูกต้อง
- Change format of day
- รวม data
- แยก data
- สร้าง fake values
- สรุปผลข้อมูล
- Check Value

## เติมช่องว่าง ย้ายข้อมูลให้ถูกต้อง และเปลี่ยน format วันที่

### canceled_order dataframe
ตัด -CN ที่อยู่ท้าย invoice_number ไปลงข้อมูลใน status แทน

In [151]:
# remove "-CN" from invoice_number column and add "Canceled" to new status column
df["canceled_order"] = df["canceled_order"].with_columns(
    pl.lit("Canceled").alias("status"),
    pl.col("invoice_number").str.replace("-CN","")
)
df["canceled_order"]

['cancel_id',
 'updated_at',
 'order_id',
 'invoice_number',
 'order_number',
 'sales_channel',
 'require_vat',
 'shipping_charge',
 'discount_baht',
 'name',
 'billing_address',
 'shipping_address',
 'phone',
 'cancel_reason',
 'status']

### Other
- เติมข้อมูลที่อยู่ ถ้าที่อยู่จัดส่งหรือที่อยู่ตามหน้าบิล อันใดอันหนึ่งเป็น null ให้ใช้ข้อมูลเหมือนที่อยู่อีกอันได้เลย
- แปลง format ของวันที่

In [177]:
for df_name in csv_list:
  for column in df[df_name]:
    if column.name in ["billing_address", "shipping_address"]:
      df[df_name] = df[df_name].with_columns(
        pl.col("billing_address").fill_null(pl.col("shipping_address")),
        pl.col("shipping_address").fill_null(pl.col("billing_address"))
      )
    if "_at" in column.name and column.dtype != pl.Datetime:
      print(f"Table: {df_name}")
      print(f"Column: {column.name}")
      df[df_name] = df[df_name].with_columns(
        pl.coalesce(
          pl.col(column.name).str.strptime(pl.Datetime, "%d/%m/%Y %H:%M:%S", strict=False),
          pl.col(column.name).str.strptime(pl.Date, "%d/%m/%Y", strict=False)
        )
      )
      print("--------")


## รวม data
- sale_order + cn_order(canceled order) -> sale_order
- inventory_output + cn_item(canceled item) + Price -> order_item

## แยก data
- awb_info -> waybill + customer_address(1)
  > เอา platform ออกเพราะมี sales_channel ใน sale_order แล้ว
- customer_code -> customer + customer_category + customer_address(2)
  >customer code/address/... -> customer_address_id

In [ ]:
.drop_duplicates()


## Use Faker library to generate fake names

In [ ]:
!pip install faker

In [ ]:
from faker import Faker
fake = Faker()
customer_master['Name'] = customer_master['CustomerNo'].apply(lambda x: fake.name())

## Summary

In [ ]:
df.info()

In [ ]:
df.describe()

# Step 3) Insert into Database

Insert ข้อมูลลงใน MySQL Database
- อ่าน Secrets ของ Colab (เก็บลง class Config)
- ใช้ Sqlalchemy เชื่อมต่อไปที่ MySQL
- สร้าง Schema (database)
- Insert ข้อมูลลงใน tables

## Getting Secrets

In [ ]:
from google.colab import userdata

class Config:
  MYSQL_HOST = userdata.get("MYSQL_HOST")
  MYSQL_PORT = userdata.get("MYSQL_PORT")
  MYSQL_USER = userdata.get("MYSQL_USER")
  MYSQL_PASSWORD = userdata.get("MYSQL_PASSWORD")
  MYSQL_DB = ''
  MYSQL_CHARSET = 'utf8mb4'


## Connect to MySQL

In [ ]:
!pip install pymysql

In [ ]:
import sqlalchemy

engine = sqlalchemy.create_engine(
    "mysql+pymysql://{user}:{password}@{host}:{port}/{db}".format(
        user=Config.MYSQL_USER,
        password=Config.MYSQL_PASSWORD,
        host=Config.MYSQL_HOST,
        port=Config.MYSQL_PORT,
        db=Config.MYSQL_DB,
    )
)

In [ ]:
df

**Tables:**
- transaction
- customer
- product

## Create Schema

In [ ]:
schema_name = "elgo"

with engine.connect() as connection:
    connection.execute(sqlalchemy.text(f"CREATE DATABASE IF NOT EXISTS {schema_name};"))


## Insert to tables

In [ ]:
df.to_sql(
    '',        # Name of the table to be created
    con=engine,           # SQLAlchemy engine connection
    schema=schema_name,   # Database name
    if_exists='replace',  # 'fail', 'replace', or 'append'
    index=False,          # Whether to include the DataFrame's index as a column
    chunksize=1000,       # Number of rows to insert in each chunk (for large DataFrames)
)

536350

# Step 4) Query and save output files

Query ข้อมูลใน table และเซฟไฟล์ output
- Query ข้อมูลด้วย Pandas
- Output ไฟล์เป็น parquet

## Query

In [ ]:
sql = "SELECT * FROM "
 = pd.read_sql_query(sql, engine)

,CustomerNo,Country,Name
0,17490.0,United Kingdom,Sara Griffin
1,13069.0,United Kingdom,Michael Holt
2,12433.0,Norway,Kelli Sandoval
3,13426.0,United Kingdom,Dalton Graves
4,17364.0,United Kingdom,Michelle James
...,...,...,...
4734,16274.0,United Kingdom,Megan Young
4735,14142.0,United Kingdom,Luke Williams
4736,13065.0,United Kingdom,Lisa Jones
4737,18011.0,United Kingdom,Kelly Jenkins


## Save to Parquet files

In [ ]:
df.to_parquet(".parquet", index=False)
